In [11]:
#Import dependencies
import pandas as pd
import requests
from citipy import citipy

In [12]:
#Get the null dataframe to work on
file_path='../Resources/IslandNulls.csv'
nulls=pd.read_csv(file_path)
nulls.head(30)

,Unnamed: 0,City,State/Province,Country,Latitude,Longitude
0,2,Cook Islands,NaN,Cook Islands,NaN,NaN
1,13,Antigua and Barbuda,NaN,Antigua and Barbuda,NaN,NaN
2,16,Turks and Caicos Islands,NaN,Turks and Caicos Islands,NaN,NaN
3,29,Mauritius,NaN,Mauritius,NaN,NaN
4,31,Cayman Islands,NaN,Cayman Islands,NaN,NaN
5,32,Seychelles,NaN,Seychelles,NaN,NaN
6,47,Barbados,NaN,Barbados,NaN,NaN
7,56,Jamaica,NaN,Jamaica,NaN,NaN
8,86,Fiji,NaN,Fiji,NaN,NaN
9,89,Saint Lucia,NaN,Saint Lucia,NaN,NaN


In [13]:
#run the same lat-lng function from the other jupyter notebook on this dataframe, but drop the 'city' part of the request 
#running just country should net us the islands we want
def lat_lng(city, state, country):
    if city is None:
        return None, None
    if state is None:
        return None, None
    location=f'{city},{country}'
    response=requests.get(f"https://nominatim.openstreetmap.org/search?country={country}&format=json")
    if response.status_code==200:
        Json=response.json()
        if len(Json) > 0:
            # Extract latitude and longitude coordinates
            lat = float(Json[0]['lat'])
            lng = float(Json[0]['lon'])
            return lat, lng
        else:
            return None, None
    else:
        return None, None
nulls['Latitude'], nulls['Longitude'] = zip(*nulls.apply(lambda row: lat_lng(row['City'], row['State/Province'], row['Country']), axis=1))
nulls.head(30)


,Unnamed: 0,City,State/Province,Country,Latitude,Longitude
0,2,Cook Islands,NaN,Cook Islands,-19.997,-157.786
1,13,Antigua and Barbuda,NaN,Antigua and Barbuda,17.2235,-61.9555
2,16,Turks and Caicos Islands,NaN,Turks and Caicos Islands,21.7217,-71.5528
3,29,Mauritius,NaN,Mauritius,-20.2759,57.5704
4,31,Cayman Islands,NaN,Cayman Islands,19.7032,-79.9175
5,32,Seychelles,NaN,Seychelles,-4.6575,55.454
6,47,Barbados,NaN,Barbados,13.15,-59.525
7,56,Jamaica,NaN,Jamaica,18.1851,-77.3948
8,86,Fiji,NaN,Fiji,-18.124,179.012
9,89,Saint Lucia,NaN,Saint Lucia,13.825,-60.975


In [14]:
#Drop everything that didn't populate a geolocation & drop duplicates as well
nulls=nulls.dropna(subset=['Latitude'])
nulls.drop_duplicates(subset=['Latitude'], inplace=True)

nulls.head(30)

,Unnamed: 0,City,State/Province,Country,Latitude,Longitude
0,2,Cook Islands,NaN,Cook Islands,-19.997,-157.786
1,13,Antigua and Barbuda,NaN,Antigua and Barbuda,17.2235,-61.9555
2,16,Turks and Caicos Islands,NaN,Turks and Caicos Islands,21.7217,-71.5528
3,29,Mauritius,NaN,Mauritius,-20.2759,57.5704
4,31,Cayman Islands,NaN,Cayman Islands,19.7032,-79.9175
5,32,Seychelles,NaN,Seychelles,-4.6575,55.454
6,47,Barbados,NaN,Barbados,13.15,-59.525
7,56,Jamaica,NaN,Jamaica,18.1851,-77.3948
8,86,Fiji,NaN,Fiji,-18.124,179.012
9,89,Saint Lucia,NaN,Saint Lucia,13.825,-60.975


In [16]:
#roll that out to a csv to pick back up on other notebook
nulls.to_csv('../Resources/FixedNulls.csv',index=False)